# Lecture 4 — Application Routing Agent

**Goal**: Build a simple agentic system that routes job applications through multiple stages

## The Agent Loop
```
while not done:
    observe(candidate_features, tool_results)
    decide_action()  # LLM chooses which tool to call
    execute_tool()
    log_result()
```

In [ ]:
# Setup
import sys
from pathlib import Path
from IPython.display import display
sys.path.insert(0, str(Path.cwd()))

import json
import pandas as pd
import random
from agent_utils import (
    structured_llm_call, 
    load_job_requirements,
    TOOL_REGISTRY
)

# Load resume utilities from lecture 3
sys.path.insert(0, str(Path.cwd() / '../lecture_3/notebooks'))
from resume_utils import load_resumes

random.seed(111)

# Configuration
OPENROUTER_API_KEY = ""  # Paste your key here

if not OPENROUTER_API_KEY or OPENROUTER_API_KEY.strip() == "":
    raise RuntimeError(
        "⚠️  Please set OPENROUTER_API_KEY above before running this notebook.\n"
        "Get your key from: https://openrouter.ai/keys"
    )

print("✓ Imports loaded")
print("✓ API key configured")
print(f"\n📦 Available tools: {len(TOOL_REGISTRY)}")
print("   Tools:", ", ".join(TOOL_REGISTRY.keys()))

## Load Data

We'll use the same resumes and job requirements from Lecture 3.

In [ ]:
# Load resumes and job requirements
resumes = load_resumes('../data/resumes_final.csv')
job_req = load_job_requirements('../data/job_req_senior.md')

print(f"Loaded {len(resumes)} resumes")

# Sample 3 candidates for demonstration
resume_list = list(resumes.values())
resume_samples = random.sample(resume_list, 3)

print(f"Selected {len(resume_samples)} candidates for routing")
print("\nCandidate IDs:", [r['ID'] for r in resume_samples])

## Step 1: Extract Features (From Lecture 3)

Before routing, we need to extract key features from each resume.
This uses the decomposition technique from Lecture 3.

In [ ]:
# Feature extraction prompt (reusing Lecture 3 concepts)
feature_extraction_prompt = """
Extract key hiring features from this resume.

You MUST cite exact quotes from the resume for years of experience.
"""

feature_schema = {
    "years_experience": "number",
    "tech_skills": ["list of technologies/languages found"],
    "education_level": "string (e.g., 'Bachelor\'s', 'Master\'s', 'PhD', 'None mentioned')",
    "evidence": "string - quotes from resume supporting years_experience"
}

# Extract features for all candidates
candidate_features = []

for idx, resume in enumerate(resume_samples):
    print(f"Extracting features for candidate {idx+1}/{len(resume_samples)}...")
    
    result = structured_llm_call(
        api_key=OPENROUTER_API_KEY,
        prompt=feature_extraction_prompt,
        context_data={"resume": resume['Resume_str']},
        output_schema=feature_schema,
        temperature=0.2
    )
    
    features = result['result']
    features['candidate_id'] = resume['ID']
    candidate_features.append(features)

display(pd.DataFrame(candidate_features))

## Step 2: Understanding the Tool Registry

Before building the agent, let's examine what tools are available.

Each tool has:
- A **function** (Python code to execute)
- A **description** (what it does)
- **parameters** (what inputs it needs)

In [ ]:
# Display tool registry
print("🔧 AVAILABLE TOOLS\n" + "="*70)

for tool_name, tool_info in TOOL_REGISTRY.items():
    print(f"\n📌 {tool_name}")
    print(f"   Description: {tool_info['description']}")
    print(f"   Parameters:")
    for param, desc in tool_info['parameters'].items():
        print(f"      • {param}: {desc}")

## Step 3: Build the Agent Decision Function

The agent needs to:
1. Observe the candidate's features and previous actions
2. Decide which tool to call next (or if done)
3. Provide parameters for that tool

In [ ]:
def agent_decide_action(
    api_key: str,
    candidate_id: str,
    features: dict,
    job_requirements: str,
    action_history: list,
    tool_registry: dict,
    temperature: float,
) -> dict:
    """
    Agent decides which tool to call next based on candidate features and history.
    
    Returns:
        dict with 'tool', 'parameters', 'reasoning'
    """
    # Build tool descriptions for the agent
    tools_desc = "\n".join([
        f"- {name}: {info['description']}\n  Parameters: {json.dumps(info['parameters'], indent=4)}"
        for name, info in tool_registry.items()
    ])
    
    # Build action history string
    history_str = "\n".join([
        f"Turn {i+1}: Called '{action['tool']}' -> {action['result']['message']}"
        for i, action in enumerate(action_history)
    ]) if action_history else "No previous actions"
    
    decision_prompt = f"""
You are a hiring automation agent. Your job is to route job applications appropriately.

CANDIDATE FEATURES:
- ID: {candidate_id}
- Years of experience: {features.get('years_experience', 'unknown')}
- Tech skills: {', '.join(features.get('tech_skills', []))}
- Education: {features.get('education_level', 'unknown')}

JOB REQUIREMENTS:
- 5-10 years experience required
- Technologies: .NET, C#, JavaScript, SQL, AWS

ACTION HISTORY:
{history_str}

AVAILABLE TOOLS:
{tools_desc}

RULES:
1. Strong candidates (5+ years, good tech match) → schedule_technical_assessment
2. Borderline candidates → flag_for_manual_review or request_additional_info
3. Weak candidates (< 3 years, poor match) → reject_application
4. After scheduling assessment → send_email with appropriate template
5. When all necessary actions taken → call 'done'

Decide the NEXT action to take. Consider what's already been done.
"""
    
    decision_schema = {
        "tool": "string - name of tool to call (must be from available tools)",
        "parameters": "object - parameters for the tool (must match tool's parameter schema)",
        "reasoning": "string - explanation of why this action is appropriate"
    }
    
    result = structured_llm_call(
        api_key=api_key,
        prompt=decision_prompt,
        context_data={},
        output_schema=decision_schema,
        temperature=temperature
    )
    
    return result['result'], result['usage']

## Step 4: Build the Agent Loop

Now we implement the core agent loop:
- **Observe**: Current state and history
- **Think**: Decide next action (call LLM)
- **Act**: Execute the tool
- **Repeat**: Until 'done' or max turns reached

In [ ]:
def agent_loop(
    api_key: str,
    candidate_id: str,
    features: dict,
    job_requirements: str,
    tool_registry: dict,
    temperature: float,
    max_turns: int = 5,
    verbose: bool = True
) -> dict:
    """
    Run the agent loop for a single candidate.
    
    Returns:
        dict with 'actions' (list of all actions taken) and 'summary' (final state)
    """
    action_history = []
    total_tokens = 0
    
    if verbose:
        print(f"\n{'='*70}")
        print(f"Processing Candidate {candidate_id}")
        print(f"{'='*70}")
        print(f"Extracted Features:")
        print(f"  • Years experience: {features.get('years_experience', 'unknown')}")
        print(f"  • Tech skills: {', '.join(features.get('tech_skills', []))[:80]}...")
        print(f"  • Education: {features.get('education_level', 'unknown')}")
    
    for turn in range(1, max_turns + 1):
        if verbose:
            print(f"\n🤔 Agent Decision (Turn {turn}):")
        
        # Agent decides next action
        decision, usage = agent_decide_action(
            api_key=api_key,
            candidate_id=candidate_id,
            features=features,
            job_requirements=job_requirements,
            action_history=action_history,
            tool_registry=tool_registry,
            temperature=temperature
        )
        
        total_tokens += usage.get('total_tokens', 0)
        
        tool_name = decision.get('tool')
        params = decision.get('parameters', {})
        reasoning = decision.get('reasoning', '')
        
        if verbose:
            print(f"  ├─ Tool: {tool_name}")
            print(f"  ├─ Reasoning: {reasoning}")
            print(f"  └─ Parameters: {json.dumps(params, indent=6)}")
        
        # Validate tool exists
        if tool_name not in tool_registry:
            if verbose:
                print(f"  ❌ Error: Tool '{tool_name}' not found in registry")
            break
        
        # Execute tool
        tool_function = tool_registry[tool_name]['function']
        try:
            result = tool_function(**params)
            if verbose:
                print(f"\n  ✓ Tool Result: {result['message']}")
        except Exception as e:
            result = {"status": "error", "message": str(e)}
            if verbose:
                print(f"\n  ❌ Tool Error: {e}")
        
        # Log action
        action_history.append({
            'turn': turn,
            'tool': tool_name,
            'parameters': params,
            'reasoning': reasoning,
            'result': result,
            'tokens_used': usage.get('total_tokens', 0)
        })
        
        # Check if done
        if tool_name == 'done' or result.get('final', False):
            if verbose:
                print(f"\n✅ Complete - {turn} turns, {total_tokens:,} tokens used")
            break
    
    # Determine final outcome
    final_actions = [a['tool'] for a in action_history]
    if 'schedule_technical_assessment' in final_actions:
        outcome = 'PROCEED_TO_INTERVIEW'
    elif 'reject_application' in final_actions:
        outcome = 'REJECTED'
    elif 'flag_for_manual_review' in final_actions:
        outcome = 'MANUAL_REVIEW'
    else:
        outcome = 'IN_PROGRESS'
    
    return {
        'actions': action_history,
        'summary': {
            'candidate_id': candidate_id,
            'years_experience': features.get('years_experience'),
            'tech_skills_count': len(features.get('tech_skills', [])),
            'total_turns': len(action_history),
            'total_tokens': total_tokens,
            'final_outcome': outcome,
            'actions_taken': ', '.join(final_actions)
        }
    }

## Step 5: Run the Agent on Sample Candidates

Now let's run our agent on the 3 sampled candidates and observe the results.

In [ ]:
# Run agent on all candidates
all_actions = []
all_summaries = []

for features in candidate_features:
    result = agent_loop(
        api_key=OPENROUTER_API_KEY,
        candidate_id=features['candidate_id'],
        features=features,
        job_requirements=job_req,
        tool_registry=TOOL_REGISTRY,
        temperature=0.2,
        max_turns=5,
        verbose=True
    )
    
    # Collect results
    for action in result['actions']:
        action['candidate_id'] = features['candidate_id']
        all_actions.append(action)
    
    all_summaries.append(result['summary'])

print(f"\n\n{'='*70}")
print("✓ All candidates processed")
print(f"{'='*70}")

## Step 6: Analyze Results with DataFrames

Now let's create DataFrames to analyze the agent's decisions.

In [ ]:
# Create action log DataFrame
action_log_df = pd.DataFrame(all_actions)

# Create summary DataFrame
summary_df = pd.DataFrame(all_summaries)

print("\n📊 ACTION LOG (all tool calls):")
print("="*70)
display(action_log_df[['candidate_id', 'turn', 'tool', 'reasoning', 'tokens_used']])

print("\n\n📊 SUMMARY (final outcomes):")
print("="*70)
display(summary_df)

## Step 7: Summary Statistics

In [ ]:
print("\n📈 SUMMARY STATISTICS")
print("="*70)

print(f"\nTotal candidates processed: {len(summary_df)}")
print(f"\nOutcomes:")
for outcome, count in summary_df['final_outcome'].value_counts().items():
    print(f"  • {outcome}: {count}")

print(f"\nAgent Efficiency:")
print(f"  • Average turns per candidate: {summary_df['total_turns'].mean():.1f}")
print(f"  • Total tokens used: {summary_df['total_tokens'].sum():,}")
print(f"  • Average tokens per candidate: {summary_df['total_tokens'].mean():.0f}")

print(f"\nMost Common Tools:")
tool_counts = action_log_df['tool'].value_counts()
for tool, count in tool_counts.items():
    print(f"  • {tool}: {count}")

# To Do

## Task #0: Run everything

Do you agree with the results for the 3 candidates

## Task #1: Cost Analysis

You have completed the task for 3 candidates. 

**Questions to answer:**
- How much did it cost to process these 3 candidates? (we are using the Claude model)
- Based on this sample, how much would it cost to process the entire pile of 130 candidates?

**Hints:**
- Check the OpenRouter/Anthropic pricing for Claude 3.5 Sonnet
- Look at the total tokens used in `summary_df`
- Remember: pricing is typically per 1M tokens, split between input and output tokens

## Task #2: Temperature Experiment

We spoke about model [temperature](https://www.ibm.com/think/topics/llm-temperature).

**Questions to explore:**
- What happens when we modify the temperature? 
- It's currently set to 0.2, what happens when we go to 0.5 or 1.0?
- Try running the same 3 candidates with different temperatures and compare:
  - Do the outcomes change?
  - Does the reasoning differ?
  - Which temperature gives more consistent results?

**Hint:** Modify the `temperature=0.2` parameter in the `agent_loop` call in Step 5

## Task #3: Full Sample Run and Cost Verification

Now that you understand the costs, run the agent on the **entire dataset** of 130 candidates to verify your cost estimate.

**Steps:**
1. Extract features for ALL resumes (not just 3)
2. Run the agent on all candidates with `verbose=False` to reduce output
3. Analyze the results:
   - Total cost
   - Distribution of outcomes (REJECTED, PROCEED_TO_INTERVIEW, MANUAL_REVIEW)
   - Average tokens per candidate
   - Most common tool usage patterns
4. Compare actual cost to your estimate from Task #1

